In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import numpy as np

Merge de documentos por sha

In [2]:
data_pdf = pd.read_csv(r'..\..\data\docs-pdf.csv')
data_preprocesada = pd.read_csv(r'..\..\data\data-preprocesada-lstm.csv')
data1 = pd.read_csv(r'..\..\data\train\data-train-lstm.csv')

data1.abstract = data1.abstract.astype(str)
data1.journal = data1.journal.astype(str)
data_preprocesada.abstract = data_preprocesada.abstract.astype(str)
data_preprocesada.journal = data_preprocesada.journal.astype(str)
data_pdf.full_text = data_pdf.full_text.astype(str)

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stopwords = set(stopwords.words('english'))

def conteo_total(text):
    n = 0

    for word in text.split():
        n+=1

    return n

def conteo_stopwords(text):
    n = 0
    
    for word in text.split():
        if word in stopwords:
            n+=1

    return n

def cadena_stopwords(text):
    text = ' '.join([word for word in text.split() if word in stopwords])
    return text
    
def cadena(text):
    text = ' '.join([word for word in text.split()])
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Juanma\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Recuperamos los sha del dataset

In [17]:
data_preprocesada = data_preprocesada.dropna(subset="sha")

In [35]:
data_preprocesada[data_preprocesada.sha=="d1aafb70c066a2068b02786f8929fd9c900897fb"]

,sha,abstract,journal
0,d1aafb70c066a2068b02786f8929fd9c900897fb,clinical features cultureproven mycoplasma pne...,bmc infect dis


In [26]:
data_temp = data1.merge(data_preprocesada,how="left", on=['abstract','journal'])

In [37]:
data_temp[data_temp.sha=="d1aafb70c066a2068b02786f8929fd9c900897fb"]

,abstract,journal,sha
80417,clinical features cultureproven mycoplasma pne...,bmc infect dis,d1aafb70c066a2068b02786f8929fd9c900897fb


In [40]:
data = data_temp.merge(data_pdf,how="left", on='sha')

In [41]:
data[data.sha=="d1aafb70c066a2068b02786f8929fd9c900897fb"]

,abstract,journal,sha,full_text
80417,clinical features cultureproven mycoplasma pne...,bmc infect dis,d1aafb70c066a2068b02786f8929fd9c900897fb,Mycoplasma pneumoniae is a common cause of up...


In [42]:
data.drop(["sha"],\
    inplace=True,axis=1)

Filtrado de texto

In [45]:
import re

data.full_text = data.full_text.astype(str)

data['full_text'] = data['full_text'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))
data['full_text'] = data['full_text'].apply(lambda x: x.lower())
data = data[data['full_text'].str.strip().astype(bool)]

Borrado de stopwords

In [46]:
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text

data['full_text'] = data['full_text'].apply(lambda x: remove_stopwords(x))

In [ ]:
print("Antes:",data.abstract[0])

data["abstract"] = data["full_text"] + " " + data["abstract"]
data.drop(['full_text'],inplace=True,axis=1)

print("Después:",data.abstract[0])

In [16]:
from collections import Counter

text_full = data['abstract'].apply(lambda x: cadena(x))
text_full_split = ' '.join(str(item) for item in text_full)
text_full_split = list(text_full_split.split(" "))
text_full_count = Counter(text_full_split)

text_stopwords = data['abstract'].apply(lambda x: cadena_stopwords(x))
text_stopwords_split = ' '.join(str(item) for item in text_stopwords)
text_stopwords_split = list(text_stopwords_split.split(" "))
text_stopwords_count = Counter(text_stopwords_split)

total = 0
total_stopwords = 0

for text in data['abstract']:
    total += conteo_total(text)
    total_stopwords += conteo_stopwords(text)

print("Palabras únicas:",len(text_full_count))
print("Palabras totales:",total)
print("Stopwords únicos:",len(text_stopwords_count))
print("Stopwords totales:",total_stopwords)
print("Palabras relevantes únicas:",len(text_full_count)-len(text_stopwords_count))
print("Palabras relevantes totales:",total-total_stopwords)

Palabras únicas: 1924306
Palabras totales: 208629448
Stopwords únicos: 1
Stopwords totales: 0
Palabras relevantes únicas: 1924305
Palabras relevantes totales: 208629448


In [48]:
data.to_csv(r'..\..\data\train\data-fulltext-train-lstm.csv', index=False)